In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yousefyousefmejdi","key":"ca5b3e870498f2eb08dc7cff32611370"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d anzarusmani/monuments

Dataset URL: https://www.kaggle.com/datasets/anzarusmani/monuments
License(s): unknown
100% 992M/996M [00:07<00:00, 133MB/s]
100% 996M/996M [00:07<00:00, 139MB/s]


In [ ]:
!unzip -q monuments.zip -d archive

In [ ]:
!ls archive

Monument


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
import cv2
import os
import numpy as np
import albumentations as A
from ultralytics import YOLO
from PIL import Image
import random

# Load YOLOv8 model for object detection
model = YOLO("yolov10n.pt")  # Pre-trained model

# Define damage pipeline
def apply_realistic_damage(image):
    # Random cracks
    if random.random() < 0.5:
        num_cracks = random.randint(1, 5)
        for _ in range(num_cracks):
            x1, y1 = random.randint(0, image.shape[1]), random.randint(0, image.shape[0])
            x2, y2 = random.randint(0, image.shape[1]), random.randint(0, image.shape[0])
            thickness = random.randint(1, 3)
            color = (0, 0, 0)  # Black cracks
            cv2.line(image, (x1, y1), (x2, y2), color, thickness)

    # Erosion (worn-out areas)
    if random.random() < 0.5:
        kernel_size = random.randint(3, 7)
        kernel = np.ones((kernel_size, kernel_size), np.uint8)
        image = cv2.erode(image, kernel, iterations=1)

    # Missing parts (random holes)
    if random.random() < 0.5:
        num_holes = random.randint(1, 5)
        for _ in range(num_holes):
            x1, y1 = random.randint(0, image.shape[1]), random.randint(0, image.shape[0])
            x2, y2 = x1 + random.randint(10, 50), y1 + random.randint(10, 50)
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 0), -1)  # Black rectangles

    # Color fading
    if random.random() < 0.5:
        alpha = random.uniform(0.5, 0.8)  # Fading intensity
        image = cv2.addWeighted(image, alpha, np.zeros_like(image), 1 - alpha, 0)

    # Noise and texture
    if random.random() < 0.5:
        noise = np.random.normal(0, 25, image.shape).astype(np.uint8)
        image = cv2.add(image, noise)

    return image

# Paths
train_clean_dir = "archive/Monument/Monument/train/Statue of Liberty"
train_damaged_dir = "archive/Monument/Monument/train_damaged/Statue of Liberty"
os.makedirs(train_damaged_dir, exist_ok=True)

# Process each image
for filename in os.listdir(train_clean_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Load image
        clean_path = os.path.join(train_clean_dir, filename)
        image = cv2.imread(clean_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Perform object detection
        results = model(image)

        # Find the largest bounding box (assumed to be the monument)
        largest_box = None
        largest_area = 0
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = box.xyxy[0].tolist()  # Get bounding box coordinates
                area = (x2 - x1) * (y2 - y1)  # Calculate area
                if area > largest_area:
                    largest_area = area
                    largest_box = (int(x1), int(y1), int(x2), int(y2))

        # If a monument is detected, apply damage only to the monument region
        if largest_box:
            x1, y1, x2, y2 = largest_box
            monument_region = image[y1:y2, x1:x2]  # Crop the monument region

            # Apply realistic damage to the monument region
            damaged_region = apply_realistic_damage(monument_region)

            # Replace the monument region with the damaged version
            image[y1:y2, x1:x2] = damaged_region

        # Save the damaged image
        damaged_path = os.path.join(train_damaged_dir, f"damaged_{filename}")
        Image.fromarray(image).save(damaged_path)
        print(f"Saved: {damaged_path}")


0: 480x640 (no detections), 396.1ms
Speed: 11.7ms preprocess, 396.1ms inference, 4.7ms postprocess per image at shape (1, 3, 480, 640)
Saved: archive/Monument/Monument/train_damaged/Statue of Liberty/damaged_frame2012.jpg

0: 384x640 1 person, 135.0ms
Speed: 10.5ms preprocess, 135.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Saved: archive/Monument/Monument/train_damaged/Statue of Liberty/damaged_945d620a17.jpg

0: 640x416 1 person, 130.7ms
Speed: 2.5ms preprocess, 130.7ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 416)
Saved: archive/Monument/Monument/train_damaged/Statue of Liberty/damaged_7c845b31ed.jpg

0: 640x640 (no detections), 215.6ms
Speed: 5.0ms preprocess, 215.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)
Saved: archive/Monument/Monument/train_damaged/Statue of Liberty/damaged_98ce33f1bd.jpg

0: 480x640 1 person, 109.3ms
Speed: 2.5ms preprocess, 109.3ms inference, 0.4ms postprocess per image at shape (1, 3, 480